In [2]:
import h5py

In [6]:
import numpy as np

In [3]:
f = h5py.File('1.mat','r') 

In [4]:
normals = f['normals']

In [7]:
normals = np.array(normals)

In [12]:
normals = normals.transpose(1,2,0)

In [6]:
%save temp
normals.shape

'' was not found in history, as a file, url, nor in the user namespace.


NameError: name 'normals' is not defined

In [4]:
norms =  np.linalg.norm(normals,axis=2).reshape(normals.shape[0],normals.shape[1],1)

!ls
!ls -la
%history
%history
norms =  np.linalg.norm(normals,axis=2).reshape(normals.shape[0],normals.shape[1],1)


NameError: name 'np' is not defined

In [35]:
import sklearn

In [37]:
from sklearn import preprocessing

In [46]:
norms[np.where(norms==0)]=1e-12

In [50]:
normals = normals / norms

In [56]:
normals[np.where(normals != 1)].shape

(718641,)

In [3]:
%history

!ls
!ls -la
%history


In [5]:
%save test

'' was not found in history, as a file, url, nor in the user namespace.


In [3]:
%%writefile ../Code/random_rotation.py
# %load ../Code/random_rotation.py
Boo!
import scipy.ndimage as ndi

def random_rotation(x, rg, row_axis=0, col_axis=1, channel_axis=2,
                    fill_mode='nearest', cval=0., order=3):
    """Performs a random rotation of a Numpy image tensor.
    # Arguments
        x: Input tensor. Must be 3D.
        rg: Rotation range, in degrees.
        row_axis: Index of axis for rows in the input tensor.
        col_axis: Index of axis for columns in the input tensor.
        channel_axis: Index of axis for channels in the input tensor.
        fill_mode: Points outside the boundaries of the input
            are filled according to the given mode
            (one of `{'constant', 'nearest', 'reflect', 'wrap'}`).
        cval: Value used for points outside the boundaries
            of the input if `mode='constant'`.
        order: The order of the spline interpolation, default is 3. 
            The order has to be in the range 0-5.
    # Returns
        Rotated Numpy image tensor.
    """
    theta = np.pi / 180 * np.random.uniform(-rg, rg)
    rotation_matrix = np.array([[np.cos(theta), -np.sin(theta), 0],
                                [np.sin(theta), np.cos(theta), 0],
                                [0, 0, 1]])

    h, w = x.shape[row_axis], x.shape[col_axis]
    transform_matrix = transform_matrix_offset_center(rotation_matrix, h, w)
    x = apply_transform(x, transform_matrix, channel_axis, fill_mode, cval, order)
    return x

def transform_matrix_offset_center(matrix, x, y):
    o_x = float(x) / 2 + 0.5
    o_y = float(y) / 2 + 0.5
    offset_matrix = np.array([[1, 0, o_x], [0, 1, o_y], [0, 0, 1]])
    reset_matrix = np.array([[1, 0, -o_x], [0, 1, -o_y], [0, 0, 1]])
    transform_matrix = np.dot(np.dot(offset_matrix, matrix), reset_matrix)
    return transform_matrix

def apply_transform(x,
                    transform_matrix,
                    channel_axis=0,
                    fill_mode='nearest',
                    cval=0.,
                    order = 3):
    """Apply the image transformation specified by a matrix.
    # Arguments
        x: 2D numpy array, single image.
        transform_matrix: Numpy array specifying the geometric transformation.
        channel_axis: Index of axis for channels in the input tensor.
        fill_mode: Points outside the boundaries of the input
            are filled according to the given mode
            (one of `{'constant', 'nearest', 'reflect', 'wrap'}`).
        cval: Value used for points outside the boundaries
            of the input if `mode='constant'`.
        order: The order of the spline interpolation, default is 3. 
            The order has to be in the range 0-5.
    # Returns
        The transformed version of the input.
    """
    x = np.rollaxis(x, channel_axis, 0)
    final_affine_matrix = transform_matrix[:2, :2]
    final_offset = transform_matrix[:2, 2]
    channel_images = [ndi.interpolation.affine_transform(
        x_channel,
        final_affine_matrix,
        final_offset,
        order=order,
        mode=fill_mode,
        cval=cval) for x_channel in x]
    x = np.stack(channel_images, axis=0)
    x = np.rollaxis(x, 0, channel_axis + 1)
    return x

Overwriting ../Code/random_rotation.py
